In [1]:
import glob
import os
import sys
sys.path.append("..")
import pandas as pd
from pandas.io.parsers import read_csv

from utils.configuration import Configuration

config = Configuration()
def add_file_name(csv, path):
    file_name = os.path.basename(path).split('.')[0]
    csv['config'] = file_name.split('result_')[1]
    return csv

In [2]:
def read_results():
    desc_map_files = []
    for path in glob.glob('../'+config.results_dir + "/*.csv"):
        csv = read_csv(path, encoding='latin-1')
        csv = add_file_name(csv, path)
        desc_map_files.append(csv)
    results= pd.concat(desc_map_files, axis=0, ignore_index=True)
    results = results[~results['target_app'].str.contains('a6|a7|a8')]
    results.drop('target_app', inplace=True, axis=1)
    results.drop('src_app', inplace=True, axis=1)
    results.fillna(0, inplace=True)
    return results

In [3]:
def config_average_performance(results):
    return results.groupby('config').mean().sort_values(by = ['accuracy'], ascending=False)

In [4]:
results = read_results()
agg_results = config_average_performance(results).sort_values(by = ['f1_score'], ascending=False)

In [5]:
# config_stat = read_csv('../'+config.analyse_dir + '/configs_status.csv')
# config_stat.set_index('config', inplace=True)
# final_result = config_stat.join(agg_results)
final_result = agg_results
final_result

,tp,tn,fp,fn,effort,accuracy,precision,recall,f1_score,reduction
config,,,,,,,,,,
use_standard_atm_atm,2.775862,0.551724,1.379310,0.862069,1.896552,0.720178,0.733025,0.787356,0.750660,0.601724
wm_blogs_custom_atm,2.557692,0.596154,1.500000,1.038462,2.019231,0.685517,0.687912,0.738782,0.704680,0.549359
jaccard_empty_custom_atm,2.568627,0.549020,1.333333,1.058824,1.960784,0.672269,0.677148,0.726797,0.695457,0.570261
nnlm_standard_custom_atm,2.603774,0.528302,1.452830,1.056604,2.037736,0.654790,0.687107,0.726101,0.693993,0.554403
use_standard_custom_craftdroid,2.553571,0.571429,1.500000,1.071429,1.982143,0.655082,0.683220,0.718750,0.691531,0.550496
use_standard_custom_atm,2.584906,0.603774,1.490566,1.075472,2.150943,0.649166,0.693239,0.719182,0.689305,0.517715
jaccard_empty_craftdroid_atm,2.470588,0.607843,1.450980,1.137255,1.921569,0.642392,0.670822,0.702288,0.673530,0.564379
wm_standard_custom_atm,2.428571,0.589286,1.607143,1.232143,2.214286,0.626978,0.647336,0.684524,0.654197,0.508036
wm_googleplay_atm_atm,2.338983,0.559322,1.525424,1.288136,2.135593,0.639969,0.660391,0.670904,0.654129,0.530791


In [6]:
configs_mrr = read_csv('../' + config.analyse_dir + '/mrr.csv')
configs_mrr['config'] = configs_mrr[['word_embedding', 'training_set', 'algorithm','descriptors']].agg('_'.join, axis=1)
configs_mrr.drop(columns=['descriptors','word_embedding', 'training_set', 'algorithm'], inplace=True)
configs_mrr.set_index('config', inplace=True)
final_result = final_result.join(configs_mrr).sort_values(by = ['f1_score'], ascending=False)
# configs_mrr.loc[configs_mrr['config'] == 'glove_android_atm_union']
final_result.to_csv('../' + config.analyse_dir + '/final_results.csv')